In [1]:
import os
import pickle
import sys

import numpy as np
import torch
import yaml
import json
import pandas as pd

# ChromaDB imports
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from typing import Dict, List, Optional, Tuple, Union

try:
    # This will work in scripts where __file__ is defined
    current_dir = os.path.dirname(os.path.abspath(__file__))
    # Assuming "src" is parallel to the script folder
    project_root = os.path.abspath(os.path.join(current_dir, ".."))
except NameError:
    # In notebooks __file__ is not defined: assume we're in notebooks/
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from main.node_embedding_models import GraphSAGE, GraphSAGE_V2
from main.ollama_utils import get_ollama_embedding
from main.sage_utils import get_new_sage_embedding


## Retrieval

In [2]:
bundle_directory = "bsard_V2_3hop_15epochs_1024-896-768-640"

# Load Graph
graph_path = os.path.join(
    project_root, "data", "retrieval_bundles", bundle_directory, "graph.pkl"
    )
with open(graph_path, "rb") as f:
    graph = pickle.load(f)

# Load sage config
config_file_path = os.path.join(
    project_root, "data", "retrieval_bundles", bundle_directory, "config.yaml"
    )
with open(config_file_path, "r") as f:
    config = yaml.safe_load(f)


# Load SAGE model
sage_model_path = os.path.join(
    project_root, "data", "retrieval_bundles", bundle_directory, "graphsage.pth"
    )

# Define model and load weights
model = GraphSAGE_V2(
    channels=config["model_params"]["channels"],   
    )
try:
    model.load_state_dict(torch.load(sage_model_path))
except:
    model.load_state_dict(torch.load(sage_model_path, map_location=torch.device('cpu')))
model.eval() 

# Set device for model inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model = model.to(device)

# Load queries
queries_path = os.path.join(
    project_root, "data", "BSARD_dataset", "bsard_train_questions.csv"
    )

queries_df = pd.read_csv(queries_path)

In [3]:
#queries_subset_df = queries_df.head(10)
queries_subset_df = queries_df


queries = list(queries_subset_df['question'])

In [4]:
def parallel_retrieval_pipeline(graph, queries, sage_model, semantic_embedder=get_ollama_embedding):

    # Get queries semantic embedding
    query_semantic_embeddings = []
    for query in queries:
        query_semantic_embeddings.append(semantic_embedder(query)['embedding'] )
    
    # Get queries sage embedding
    query_sage_embeddings = []
    for query in query_semantic_embeddings:
        query_sage_embeddings.append(get_new_sage_embedding(model=sage_model, new_feature=torch.tensor(query, dtype=torch.float32).reshape(model.convs[0].in_channels), device=device))

    # Create two chromadb collections
    chroma_client = chromadb.Client()

        # One for the semantic embeddings
    collection_semantic = chroma_client.create_collection(name="semantic_collection")
        # Fill collection
    for node_id in graph.nodes:
        node_data = graph.nodes[node_id]
        if node_data.get("node_type") == "Article":
            doc_text = node_data.get("article_text", "")
            doc_embedding = node_data.get("embedding", None)
        
            # Ensure that doc_embedding is a list (or array) of floats
            # Add the node to the collection by specifying IDs, documents, and embeddings
            collection_semantic.add(
                ids=[str(node_id)],         # The ID will be the node's ID (converted to a string)
                documents=[doc_text],         # The node's text
                embeddings=[doc_embedding]    # The precomputed embedding you already have
                )
            
        # One for the sage embeddings
    collection_sage = chroma_client.create_collection(name="sage_collection")
        # Fill collection
    for node_id in graph.nodes:
        node_data = graph.nodes[node_id]
        if node_data.get("node_type") == "Article":
            doc_text = node_data.get("article_text", "")
            doc_embedding = node_data.get("hybrid_embedding", None)
        
            # Ensure that doc_embedding is a list (or array) of floats
            # Add the node to the collection by specifying IDs, documents, and embeddings
            collection_sage.add(
                ids=[str(node_id)],         # The ID will be the node's ID (converted to a string)
                documents=[doc_text],         # The node's text
                embeddings=[doc_embedding]    # The precomputed embedding you already have
                )

    # Retrieve top k documents from the semantic collection based on the query semantic embedding for each query (based on semantic embedding)
    semantic_results = []
    for query in query_semantic_embeddings:
        topk_semantic    = collection_semantic.query(
            query_embeddings=[query] ,
            n_results=10
        )
        semantic_results.append(topk_semantic["ids"])


    # Retrieve top k documents from the sage collection based on the query sage embedding for each query (based on sage embedding)
    sage_results = []
    for query in query_sage_embeddings:
        topk_sage    = collection_sage.query(
            query_embeddings=query.detach().cpu().unsqueeze(0).tolist(),
            n_results=10
        )
        sage_results.append(topk_sage["ids"])

    
    return semantic_results, sage_results

In [5]:
semantic_results, sage_results = parallel_retrieval_pipeline(graph, queries, model)

In [6]:
# semantic_results

In [7]:
# sage_results

## Evaluation

In [8]:
# queries_df
queries_subset_df

# Load bsard corpus lean
bsard_corpus_lean_path = os.path.join(
    project_root, "data", "BSARD_dataset", "bsard_corpus_lean_V2.csv"
    )
bsard_corpus_lean = pd.read_csv(bsard_corpus_lean_path)

In [9]:
article_ids_code = []
for article_ids_list in queries_subset_df['article_ids']:
    tmp = []
    for article_id in article_ids_list.split(','):
        #print(article_id)
        tmp.append(bsard_corpus_lean[bsard_corpus_lean['id'] == int(article_id)]['article_code'].values[0])
    article_ids_code.append(tmp)
    #print(tmp)

queries_subset_df['article_ids_codes'] = article_ids_code


In [10]:
queries_subset_df

,id,category,subcategory,question,extra_description,article_ids,article_ids_codes
0,1102,Travail,Travail et parentalité,Je suis travailleur salarié(e). Puis-je refuse...,Pendant la grossesse,"22225,22226,22227,22228,22229,22230,22231,2223...","[33.10.5.1, 33.10.5.2, 33.10.5.3, 33.10.5.4, 3..."
1,91,Argent,Dettes,Peut-on saisir tous mes revenus ?,"Procédures de récupération des dettes, Récupér...","5853,5854,5855","[10.0.4.82, 10.0.4.83, 10.0.4.84]"
2,474,Famille,Situation de couples,Je suis marié(e). Nous sommes mariés. Dois-je ...,Mariage,"1096,1097,1098,1108,1109,1110","[4.2.8.4, 4.2.8.5, 4.2.8.6, 4.2.8.16, 4.2.8.17..."
3,836,Logement,Location en Wallonie,Je mets un kot en location (bail de droit comm...,"Mettre un logement en location (Wallonie), Doi...","12012,12030,12031,12032,12033,12034,12035","[21.0.1.1, 21.0.2.15, 21.0.2.16, 21.0.2.17, 21..."
4,1079,Travail,Maladie - incapacité de travail,Suis-je payé pendant la procédure du trajet de...,Rupture du contrat de travail pour force majeu...,"21114,21115,21116,21117,21118,21119,21120,2112...","[33.1.4.72, 33.1.4.73, 33.1.4.74, 33.1.4.75, 3..."
...,...,...,...,...,...,...,...
881,308,Famille,Lien parents/enfants,Quel est le rôle du tuteur d'un enfant mineur ?,"Tutelle, Rôle du tuteur","1321,1322,1323,1324,1325,1326,1327,1328","[4.2.12.18, 4.2.12.19, 4.2.12.20, 4.2.12.21, 4..."
882,387,Famille,Personnes à l'autonomie fragilisée,Peut-on changer d'administrateur de biens et/o...,Administration de biens et/ou de la personne (...,1381,[4.2.13.28]
883,940,Logement,Location à Bruxelles,Mon propriétaire ne fait pas les réparations n...,"Bail de résidence principale (Bruxelles), Loye...",2494,[4.0.9.26]
884,364,Famille,Personnes à l'autonomie fragilisée,Que faire si le montant des frais et honoraire...,Administration de biens et/ou de la personne (...,1387,[4.2.13.34]


In [11]:
# include the retrieved articles into queries_df as a new column
ret_semantic = []
ret_sage = []

for i in range(len(semantic_results)):
    n_articles = len(queries_df['article_ids_codes'][i])
    ret_semantic.append(semantic_results[i][0][:n_articles])
    ret_sage.append(sage_results[i][0][:n_articles])

queries_subset_df['ret_semantic'] = ret_semantic
queries_subset_df['ret_sage'] = ret_sage

In [12]:
# Función de Recall@K para una sola fila
def recall_at_k(true_ids, pred_ids):
    true_set = set(true_ids)
    # si no hay relevantes, definimos recall=0 (o 1 si lo prefieres)
    if not true_set:
        return 0.0
    # Intersección entre lo que debería y lo que recuperaste
    hits = true_set & set(pred_ids)
    return len(hits) / len(true_set)

# Aplicamos al DataFrame
queries_subset_df['recall_semantic'] = queries_subset_df.apply(
    lambda row: recall_at_k(row['article_ids_codes'], row['ret_semantic']),
    axis=1
)
queries_subset_df['recall_sage'] = queries_subset_df.apply(
    lambda row: recall_at_k(row['article_ids_codes'], row['ret_sage']),
    axis=1
)

In [13]:
macro_recall_semantic = queries_subset_df['recall_semantic'].mean()
print("Macro recall semantic: " ,macro_recall_semantic)
macro_recall_sage     = queries_subset_df['recall_sage'].mean()
print("Macro recall sage: " ,macro_recall_sage)

Macro recall semantic:  0.09967588011287074
Macro recall sage:  0.003466623669783941


In [20]:
queries_subset_df.iloc[0]['article_ids_codes']  

['33.10.5.1',
 '33.10.5.2',
 '33.10.5.3',
 '33.10.5.4',
 '33.10.5.5',
 '33.10.5.6',
 '33.10.5.7',
 '33.10.5.8',
 '33.10.5.9',
 '33.10.5.10']

In [21]:
queries_subset_df.head(10)

,id,category,subcategory,question,extra_description,article_ids,article_ids_codes,ret_semantic,ret_sage,recall_semantic,recall_sage
0,1102,Travail,Travail et parentalité,Je suis travailleur salarié(e). Puis-je refuse...,Pendant la grossesse,"22225,22226,22227,22228,22229,22230,22231,2223...","[33.10.5.1, 33.10.5.2, 33.10.5.3, 33.10.5.4, 3...","[33.10.5.9, 14.2.0.48, 14.2.0.50, 14.2.0.32, 2...","[16.8.4.195, 10.6.0.364, 16.8.4.239, 16.8.4.23...",0.100000,0.0
1,91,Argent,Dettes,Peut-on saisir tous mes revenus ?,"Procédures de récupération des dettes, Récupér...","5853,5854,5855","[10.0.4.82, 10.0.4.83, 10.0.4.84]","[16.6.7.334, 16.3.3.12, 4.2.6.65]","[16.8.4.195, 10.6.0.364, 16.8.4.239]",0.000000,0.0
2,474,Famille,Situation de couples,Je suis marié(e). Nous sommes mariés. Dois-je ...,Mariage,"1096,1097,1098,1108,1109,1110","[4.2.8.4, 4.2.8.5, 4.2.8.6, 4.2.8.16, 4.2.8.17...","[4.2.8.2, 4.2.8.29, 4.2.8.10, 4.2.8.20, 4.2.8....","[16.8.4.195, 10.6.0.364, 16.8.4.239, 16.8.4.23...",0.000000,0.0
3,836,Logement,Location en Wallonie,Je mets un kot en location (bail de droit comm...,"Mettre un logement en location (Wallonie), Doi...","12012,12030,12031,12032,12033,12034,12035","[21.0.1.1, 21.0.2.15, 21.0.2.16, 21.0.2.17, 21...","[21.0.3.39, 17.13.0.6, 21.0.2.16, 21.0.3.36, 2...","[16.8.4.195, 10.6.0.364, 16.8.4.238, 16.8.4.23...",0.285714,0.0
4,1079,Travail,Maladie - incapacité de travail,Suis-je payé pendant la procédure du trajet de...,Rupture du contrat de travail pour force majeu...,"21114,21115,21116,21117,21118,21119,21120,2112...","[33.1.4.72, 33.1.4.73, 33.1.4.74, 33.1.4.75, 3...","[33.1.4.81, 33.1.4.77, 29.4.2.21, 16.6.10.4, 3...","[16.8.4.195, 10.6.0.364, 16.8.4.239, 16.8.4.23...",0.363636,0.0
5,850,Logement,Location à Bruxelles,Quand faire l'état des lieux à Bruxelles ?,"Bail de résidence principale (Bruxelles), Etat...",838,[3.0.11.6],[4.0.9.33],[16.8.4.195],0.000000,0.0
6,176,Argent,Succession et donation,Mon cohabitant légal est décédé. Quels sont me...,Succession et situations de couples,"1676,2334","[4.4.2.34, 4.0.6.7]","[4.0.6.8, 4.4.2.34]","[16.8.4.195, 10.6.0.364]",0.500000,0.0
7,342,Famille,Obligations alimentaires,Je suis parent. Dois-je encore payer une contr...,Les contributions alimentaires (pour les enfants),"1055,1274","[4.2.6.47, 4.2.11.1]","[4.2.8.64, 4.2.6.59]","[16.8.4.195, 10.6.0.364]",0.000000,0.0
8,1044,Protection sociale,Aide du CPAS,Le CPAS peut-il faire des retenues sur mon RIS ?,Le revenu d'intégration sociale (RIS),5732,[10.0.2.198],[28.2.1.12],[16.8.4.195],0.000000,0.0
9,946,Logement,Location à Bruxelles,Quel bail est considéré comme un bail 9 ans à ...,"Bail de résidence principale (Bruxelles), Réda...","856,857,611,614","[3.0.11.24, 3.0.11.25, 3.0.1.2, 3.0.3.2]","[3.0.11.27, 4.0.9.46, 3.0.11.54, 4.0.9.18]","[16.8.4.195, 10.6.0.364, 16.8.4.238, 16.8.4.239]",0.000000,0.0


In [15]:
#def recall_at_k(true_ids, pred_ids, k):
#    """Recall@K para UNA sola consulta."""
#    true_set = set(true_ids)
#    topk = set(pred_ids[:k])
#    return len(true_set & topk) / len(true_set)
#
#
#def average_precision(true_ids, pred_ids, k=None):
#    """Average Precision para UNA sola consulta."""
#    if k is not None:
#        pred_ids = pred_ids[:k]
#    true_set = set(true_ids)
#    hits = 0
#    sum_prec = 0.0
#    for i, pid in enumerate(pred_ids, start=1):
#        if pid in true_set:
#            hits += 1
#            sum_prec += hits / i
#    # normalizamos por el total de relevantes
#    return sum_prec / len(true_set) if true_set else 0.0
#
#
#def r_precision(true_ids, pred_ids):
#    """Precision@R para UNA sola consulta."""
#    R = len(true_ids)
#    return recall_at_k(true_ids, pred_ids, R)
#
#
## --- Cálculo agregado sobre todo el testset ---------------------
#
## supongamos:
##   q2true[q] = lista de IDs relevantes para la pregunta q
##   q2pred[q] = lista de IDs devuelta por tu sistema RAG (ordenada por similitud)
#
#recalls, aps, rps = [], [], []
#
#for q in q2true:
#    true_ids = q2true[q]
#    pred_ids = q2pred[q]
#
#    recalls.append(recall_at_k(true_ids, pred_ids, k=10))
#    aps.append(average_precision(true_ids, pred_ids))
#    rps.append(r_precision(true_ids, pred_ids))
#
#mean_recall10 = sum(recalls) / len(recalls)
#map_score      = sum(aps)     / len(aps)
#mean_rp        = sum(rps)     / len(rps)

#print(f"Recall@10: {mean_recall10:.3f}")
#print(f"MAP:        {map_score:.3f}")
#print(f"mRP:        {mean_rp:.3f}")


In [23]:
import subprocess

with open("requirements.txt", "w") as f:
    subprocess.run(["pip", "freeze"], stdout=f)
